In [3]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

__all__ = ['ResNet', 'resnet18', 'resnet34', 'resnet50', 'resnet101', 'resnet152']


model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth'
}

In [5]:
def conv3x3(in_planes, out_planes, stride = 1):
  return nn.Conv2(in_planes, out_planes, kernel_size = 3, stride=stride, padding = 1, bias = False)

def conv1x1(in_planes, out_planes, stride = 1):
  return nn.Conv2(in_planes, out_planes, kernel_size = 3, stride=stride, bias = False)

In [9]:
class BasicBlock(nn.Module):
  expansion = 1

  def __init__(self, in_planes, planes, stride = 1, downsample = None):
    super(BasicBlock, self).__init__()
    self.conv1 = conv3x3(in_planes, planes, stride = 1)
    self.bn1 = nn.BatchNorm2d(planes)
    self.relu = nn.ReLU(inplace = True)
    self.conv2 = conv3x3(in_planes, planes)
    self.bn2 = nn.BatchNorm2d(planes)
    self.downsample = downsample
    self.stride = stride

  def forward(self, x):
    identity = x
    
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)

    if (self.downsample is not None):
      identity = self.downsample(x)

    out += identity
    out = self.relu(out)

    return out


In [11]:
class Bottleneck(nn.Module):
  expansion = 4

  def __init__(self, in_planes, planes, stride = 1, downsample = None):
    super(Bottleneck, self).__init__()
    self.conv1 = conv3x3(in_planes, planes, stride = 1)
    self.bn1 = nn.BatchNorm2d(planes)
    self.relu = nn.ReLU(inplace = True)
    self.conv2 = conv3x3(in_planes, planes)
    self.bn2 = nn.BatchNorm2d(planes)
    self.conv3 = conv1x1(planes, planes*self.expansion)
    self.bn3 = self.bn2 = nn.BatchNorm2d(planes*self.expansion)
    self.downsample = downsample
    self.stride = stride

  def forward(self, x):
    identity = x
    
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)

    out = self.conv3(out)
    out = self.bn3(out)

    if (self.downsample is not None):
      identity = self.downsample(x)

    out += identity
    out = self.relu(out)

    return out

In [ ]:
from torch.nn.modules.batchnorm import BatchNorm2d
class ResNet(nn.Module):

  def __init__(self, block, layers, num_classes = 1000, zero_init_redisual = False):
    super(ResNet, self).__init__()
    self.inplains = 64
    self.conv1 = nn.Conv2d(3, 64, kernel_size = 7, stride = 2, padding = 3, bias = False)
    self.bn1 = nn.BatchNorm2d(64)
    self.relu = nn.ReLU(inplace = True)
    self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    self.layer1 = self._make_layer(block, 64, layers[0])
    self.layer2 = self._make_layer(block, 128, layers[1], stride = 2)
    self.layer3 = self._make_layer(block, 256, layers[2], stride = 2)
    self.layer4 = self._make_layer(block, 512, layers[3], stride = 2)
    self.avgpool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(512 * block.expansion, num_classes)

    for m in self.modules():
      if (isinstance(m, nn.Conv2d)):
        nn.init.kaiming_normal_(m.weight, mode = 'fan_out', nonlinearity = 'relu')
      elif (isinstance(m, nn.BatchNorm2d)):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)

    if zero_init_redisual:
       for m in self.modules():
         if isinstance(m, Bottleneck):
           nn.init.constant_(m.bn3.weight, 0)
         elif isinstance(m, BasicBlock):
           nn.init.constant_(m.bn2.weight, 0)


  def _make_layer(self, block, planes, blocks, stride = 1):
    downsample = None
    if (stride != 1) or (self.inplanes != planes * block.expansion):
      downsample = nn.Sequential(
          conv1x1(self.inplanes, planes* block.expansion, stride),
          nn.BatchNorm2d(planes* block.expansion)
      )

    layers = []
    layers.append(block(self.inplanes, planes, stride, downsample))
    self.inplanes = planes * block.expansion
    for _ in range (1, blocks):
      layers.append(block(self.inplanes, planes))

    return nn.Sequential(*layers)

  def forward(self, x):
    x = self.conv1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.maxpool(x)

    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.layer4(x)

    x = self.avgpool(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)

    return x
